### Initialize session

In [32]:
from neo4j.v1 import GraphDatabase, basic_auth
driver = GraphDatabase.driver("bolt://localhost:7687", 
                    auth=basic_auth("neo4j", "admin"))
session = driver.session()

### Close session

In [ ]:
session.close()

### Hello world

In [ ]:
session.run("CREATE (a:Person {name:'Arthur', "
                              "title: 'King'})")
result = session.run("MATCH (a:Person) "
            "WHERE a.name = 'Arthur' "
            "RETURN a.name AS name, a.title AS title")
for record in result:
    print("%s %s" % (record['title'], record['name']))

### Query all

In [ ]:
session.run("MATCH (n) RETURN n") 

### Delete all

In [ ]:
session.run("MATCH (n) DETACH DELETE n") 

### Run statements

In [ ]:
with session.begin_transaction() as tx:
    tx.run("CREATE (:Person {name: 'Guinevere'})")
    tx.success = True

### Roll back a transaction

In [ ]:
with session.begin_transaction() as tx:
    tx.run("CREATE (:Person {name: 'Merlin'})")
    tx.success = False

### Auto-commit transactions and query parameters

In [ ]:
result = session.run("CREATE (person:Person {name: {name}})", 
                     {"name": "Arthur"})

### Result cursor

In [ ]:
search_term = "Sword"
result = session.run("MATCH (weapon:Weapon) "
                     "WHERE weapon.name CONTAINS {term} "
                     "RETURN weapon.name", 
                     {"term": search_term})
print("List of weapons called %r:" % search_term)
for record in result:
    print(record["weapon.name"])

### The record

In [33]:
search_term = "Arthur"
result = session.run("MATCH (weapon:Weapon) "
                     "WHERE weapon.owner CONTAINS {term} "
                     "RETURN weapon.name, weapon.material, weapon.size", 
                     {"term": search_term})
print("List of weapons owned by %r:" % search_term)
for record in result:
    print(", ".join("%s: %s" % 
            (key, record[key]) for key in record.keys()))

List of weapons owned by 'Arthur':


### Retaining results for further processing

In [ ]:
session = driver.session()
result = session.run("MATCH (knight:Person:Knight) "
                     "WHERE knight.castle = {castle} "
                     "RETURN knight.name AS name", 
                     {"castle": "Camelot"})
retained_result = list(result)
session.close()
for record in retained_result:
    print("%s is a knight of Camelot" % record["name"])

### Use results in a new query

In [ ]:
result = session.run("MATCH (knight:Person:Knight) "
                     "WHERE knight.castle = {castle} "
                     "RETURN id(knight) AS knight_id", 
                     {"castle": "Camelot"})
for record in result:
    session.run("MATCH (knight) "
                "WHERE id(knight) = {id} "
                "MATCH (king:Person) "
                "WHERE king.name = {king} "
                "CREATE (knight)-[:DEFENDS]->(king)", 
                {"id": record["knight_id"], 
                 "king": "Arthur"})